In [5]:
import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d
import laspy

import random
import math

In [6]:
# GET GROUND HEIGHT

las = laspy.read('data/banska4.las')
ground_data = las[las.classification == 2] 
points_data = np.stack([ground_data.Z], axis = 0).transpose((1,0))

points_sum = 0
data_min = min(points_data)

for i in range(len(points_data)):
    points_sum += points_data[i][0] - data_min[0]

avg_ground_height = math.floor(points_sum / len(points_data) + data_min[0])

In [7]:
# GET TREE EPS

tree_data = las[las.classification == 5]
points_data = np.stack([tree_data.Z], axis = 0).transpose((1,0))

points_data = sorted(points_data, key=lambda x: x[0], reverse=True)
points_data = points_data[:100]

points_sum = 0
data_min = min(points_data)

for i in range(len(points_data)):
    points_sum += points_data[i][0] - data_min[0]

avg_tree_height = math.floor(points_sum / len(points_data) + data_min[0])
tree_eps = (avg_tree_height - avg_ground_height) // 4

In [22]:
# FIRST CLUSTERING

points_data = np.stack([tree_data.X,tree_data.Y,tree_data.Z], axis = 0).transpose((1,0))
geom = o3d.geometry.PointCloud()
geom.points = o3d.utility.Vector3dVector(points_data)
geom = geom.remove_radius_outlier(3,1000)[0]

labels = np.array(geom.cluster_dbscan(tree_eps, 6))
random_colors = []
for label in labels:
    if label == -1:
        random_colors.append([0,0,0])
    else:    
        random_colors.append([random.random()*0.8 +0.2,random.random()*0.8 +0.2,random.random()*0.8 +0.2])
colors = [random_colors[x] for x in labels]
geom.colors = o3d.utility.Vector3dVector(colors)
o3d.visualization.draw_geometries([geom])

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, -1}


In [23]:
# GET SIZE OF EACH CLUSTER

cluster_sizes = []
for i in set(labels):
   cluster_sizes.append((i, labels.tolist().count(i)))
#plt.figure()
#plt.plot([x for x in range(max(labels)+2)],[x[1] for x in cluster_sizes])
#plt.show()

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, -1}


In [28]:
# GET SEPARATE WELL AND SHITTY CLUSTERED POINTS WITH LABELS

clusters_sorted = sorted(cluster_sizes,key=lambda x: x[1],reverse=True)
clusters_sizes_sorted = [x[1] for x in clusters_sorted]
avg_cluster_size = sum(clusters_sizes_sorted) / len(clusters_sizes_sorted)

well_clustered_points = []
well_clustered_labels = []

shitty_clustered_points = []
shitty_clustered_labels = []

all_points = np.asarray(geom.points)

for i in range(len(all_points)):
    for cluster in clusters_sorted:
        if labels[i] == cluster[0]:
            if cluster[1] > avg_cluster_size * 5:
                shitty_clustered_points.append(all_points[i])
                shitty_clustered_labels.append(labels[i])
            else:
                well_clustered_points.append(all_points[i])
                well_clustered_labels.append(labels[i])         


pointcloud_final = o3d.geometry.PointCloud()
pointcloud_final.points = o3d.utility.Vector3dVector(well_clustered_points)
#o3d.visualization.draw_geometries([pointcloud_final])
